## Relevant Links
- Examples: https://docs.google.com/spreadsheets/d/14pdiODt9PmsD2F___Lpv2rO-RByTtC8PYcEfxL8iIuY/edit#gid=186467173
- LangChain: https://github.com/hwchase17/langchain
- Chroma: no docker or API_KEY - https://www.trychroma.com/
- OpenAI account usage: https://platform.openai.com/account/usage

## Tasks

- **Eric & Jacob**: Validation (`validate_stages.py` is not yet working, and has not been incorporated)
- **Jacob**: Label class recognition
    - determine what classes the user is referring to for each label field
- **Allen**: Evaluation Identification Stage:
    - `links/evaluation_run_selector.py` 
    - add stage to `dataset_view_generator.py`
    - add `prompts/evaluation_task_rules.txt` prompt
    - add examples to Examples spreadsheet in new tab, and then put in `examples` folder
- **Leila**: Add support for `hardness` brain runs
    - fill out the template in `links/brain_run_selector.py` 
    - add `prompts/hardness_task_rules.txt` prompt modeled after uniqueness and mistakenness.
    - new examples tab `hardness` modeled after `uniqueness` and `mistakenness` tabs and then put in `examples` folder
- **Vini**: More examples
    - scrape examples from community Slack covering as wide a range of scenarios as possible
    - complex filters and view expressions!
    - videos
    - every type of label
    - varied naming conventions

## Additional notes

- Want to validate embedded fields
- At present, there is no support for multi-line Python code to generate views. It all needs to be done inline
- No support for groups, 3D/point clouds, `concat()`, or `mongo()`.

## Getting started

`pip install openai langchain chroma`

Then create an OpenAI account and generate an API key

`export OPENAI_API_KEY=...`

In [2]:
import fiftyone as fo
dataset = fo.load_dataset("quickstart")

## Get similar prompts

In [3]:
from links.view_stage_example_selector import generate_view_stage_examples_prompt

In [4]:
query = "Five random images from the dataset"

In [5]:
view_stage_examples_prompt = generate_view_stage_examples_prompt(dataset, query)

Using embedded DuckDB without persistence: data will be transient


In [6]:
print(view_stage_examples_prompt)

Generate code to produce the FiftyOne view stages for the following prompt:


Input: show me 50 random samples
Output: dataset.take(50)

Input: Take two random samples from the dataset
Output: dataset.take(2)

Input: 10 most unique images
Output: dataset.filter_labels("predictions", F("confidence") > 0.95)

Input: 10 random images with tables
Output: dataset.match(
    F("ground_truth.detections.label").contains("table")
).take(10)

Input: show me random samples
Output: dataset.shuffle()

Input: skip the first 10 samples
Output: dataset.skip(10)

Input: first 100 samples
Output: dataset.limit(100)

Input: Don't show me the fifth and sixth samples
Output: ids = [dataset.skip(4).first().id, dataset.skip(4).first().id];
dataset.exclude(ids)

Input: Images that only contain dogs
Output: dataset.match(
    F("ground_truth.detections.label").is_subset(["dog"])
)

Input: ten least wrong predictions
Output: dataset.sort_by("mistakenness", reverse=False)[:10]

Input: Five random images from the

## Generate View Stage Descriptions Prompt

In [7]:
from links.view_stage_description_selector import generate_view_stage_descriptions_prompt

In [8]:
view_stage_descriptions_prompt = generate_view_stage_descriptions_prompt(view_stage_examples_prompt)

In [9]:
print(view_stage_descriptions_prompt)

Here is some information about view stages you might want to use:


    View stage: skip
    Description: Omits the given number of samples from the head of the collection. Non-positive skip values result in no samples being omitted
    Inputs: skip: int

    

    View stage: take
    Description: Randomly samples the given number of samples from the collection
    Inputs: size: int (non-positive values result in an empty view), seed: optional random seed to use for sample selection

    

    View stage: match
    Description: Filters the samples in the collection by the given filter
    Inputs: filter: ViewExpression or MongoDB expression that returns a boolean describing the filter to apply

    

    View stage: exclude
    Description: Exclude samples with specified IDs
    Inputs: sample_ids: IDs to exclude. Can be single ID, iterable of IDs, Sample or SampleView instances, iterable of Sample or SampleView instances, or SampleCollection

    

    View stage: filter_labels
    D

## Brain method selector

In [7]:
from links.brain_method_selector import select_brain_methods

In [8]:
query = "Five most unique images from the dataset"
select_brain_methods(query)

['uniqueness']

In [127]:
query = "50 least unique images from the dataset that were hard"
select_brain_methods(query)

['uniqueness', 'hardness']

In [9]:
query = "Five most mistaken images from the dataset"
select_brain_methods(query)

['mistakenness']

## Brain run selector

In [10]:
from links.brain_run_selector import select_brain_runs

In [1]:
import fiftyone as fo
dataset = fo.load_dataset("quickstart")

In [12]:
query = "most unique images from the dataset with key 'uniqueness1 that are similar to image 10"
select_brain_runs(dataset, query, ["uniqueness", "image_similarity"])

{'uniqueness': 'uniqueness1', 'image_similarity': 'img_sim'}

In [16]:
query = "Five most mistaken images from the dataset with ground truth field 'predictions"
select_brain_runs(dataset, query, ["mistakenness"])

{'mistakenness': 'mistakenness'}

## Field selector

In [1]:
import fiftyone as fo
dataset = fo.load_dataset("quickstart")

In [2]:
from links.field_selector import select_fields

In [8]:
query = "sort pred1 by number of detections"
fields = select_fields(dataset, query)
print(fields)

[id: string, filepath: string, tags: list, ground_truth: Detections, predictions: Detections, classif: Classification, mistakenness1_eval_fp: int, pred1: Detections]
[pred1]


## DatasetView generator

In [1]:
from gpt_view_generator import get_gpt_view_text

In [2]:
import fiftyone as fo
dataset = fo.load_dataset("quickstart")

In [3]:
query = "images with tables"
view_text = get_gpt_view_text(dataset, query)
print(view_text)

Using embedded DuckDB without persistence: data will be transient


Finding similar examples for query: images with tables
Identified likely view stages: ['sort_by_similarity', 'limit', 'match', 'skip', 'exclude']
Identified brain methods: ['text_similarity']
Identified brain runs: {'text_similarity': 'qdrant_gt_patches'}
Identified fields: [ground_truth, predictions]
[match(
    F("ground_truth.detections.label").contains("table")
)]


In [4]:
query = "patches of rabbits with high confidence"
view_text = get_gpt_view_text(dataset, query)
print(view_text)

Using embedded DuckDB without persistence: data will be transient


Finding similar examples for query: patches of rabbits with high confidence
Identified likely view stages: ['filter_labels', 'exists', 'sort_by_similarity', 'match', 'select_fields']
Identified brain methods: ['mistakenness']
Identified brain runs: {'mistakenness': 'mistakenness5'}
Identified fields: [predictions]
[filter_labels("predictions", F("label") == "rabbit", F("confidence") > 0.95)]


In [5]:
query = "first 30 images with a ground truth detection"
view_text = get_gpt_view_text(dataset, query)
print(view_text)

Using embedded DuckDB without persistence: data will be transient


Finding similar examples for query: first 30 images with a ground truth detection
Identified likely view stages: ['match', 'sort_by_similarity', 'limit', 'limit_labels', 'skip']
Identified brain methods: []
Identified brain runs: {}
Identified fields: [ground_truth]
[match(F("ground_truth.detections").length() > 0), limit(30)]


## Roadmap

### Validation and error handling
- even fully fleshed out, this is not going to be 100% accurate. Need to validate that it is actually creating a valid DataSetView
- if it isn't, prompt the user for more specific information
    - this could potentially be tailored to which part of the process it failed

### Plugin
- Everything will be wrapped in a single python function `generate_datasetview_with_chatgpt(dataset, prompt)`
- Then need to turn this into a plugin. It will take the `session.dataset` as dataset, and will set `session.view`
- This will probably be a menu-item plugin - we could use the ChatGPT symbol - it will take user input
- Would love to have a toggle the user can specify for whether they want 
    - the view created from scratch, or 
    - the view stages concatenated with their existing view

### Future
- Memory/chat history
- More general question-answering:
    - First stage: determine whether the user is asking a question about the entire dataset, or individual samples
    - Use ChatGPT as a dispatcher, deciding what other models/processes to invoke.
        - if it is about aggregations, then decide what FiftyOne aggregation to perform, and interpret the results
        - if it is about a single image, employ BLIPv2 or equivalent...